# <a id="0"></a> Contents
1. [Overview](#1)
1. [Importing Modules, Reading the Dataset and Defining an Evaluation Table](#3)
1. [Defining a Function to Calculate the Adjusted $R^{2}$ ](#3)
1. [Creating a Simple Regression](#4)
    1. [Let's Show the Results](#5)
1. [Examining and Adding More Features](#6)
   1. [Checking Out the Correlation Among Explanatory Variables](#7)
   1. [Complex Model - 1](#8)
   1. [Complex Model - 2](#9)
1. [Polynomial Regression](#10)  
1. [KNN Regression](#11)   
1. [Conclusion](#12)

# <a id="1"></a> Overview
<hr/>
Welcome to my Kernel! In this kernel I use various regression methods and try to predict the house prices by using them. As you can guess, there are various methods to suceed this and each method has pros and cons. I think regression is one of the most important methods because it gives us more insight about the data. When we ask ***why***, it is easier to interpret the relation between the response and explanatory variables. 

Here, I start with a very simple model and continue with more complex ones. I try to find the most useful model and during this I also use visuals to be able to understand the data better. 

If you have a question or feedback, do not hesitate to write and if you like this kernel, please <b><font color="red">do not forget to </font><font color="green">UPVOTE </font></b> 🙂 
<br/>
<img src="https://i.imgur.com/kCNAFTN.jpg?1" title="source: imgur.com" />

# <a id="2"></a> Importing Modules, Reading the Dataset and Defining an Evaluation Table
#### [Return Contents](#0)
<hr/>

In order to make some analysis, we need to set our environment up. To do this, I firstly imported some modules and read the data. The below output is the head of the data but if you want to see more details, you might try removing ***#*** signs in front of the ***df.describe()*** and ***df.info()***.  

Further, I defined an empty dataframe. This dataframe includes Mean Squared Error (MSE), R-squared and Adjusted R-squared which are the important metrics to compare different models. Having a R-squared value closer to one and smaller MSE means a better fit. In the following sections, I will fill this dataframe with my results.

In [ ]:
# Importing the modules 
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

evaluation = pd.DataFrame({'Model': [],
                           'Details':[],
                           'Mean Squared Error (MSE)':[],
                           'R-squared (training)':[],
                           'Adjusted R-squared (training)':[],
                           'R-squared (test)':[],
                           'Adjusted R-squared (test)':[]})

# Reading Data 
df = pd.read_csv('../input/kc_house_data.csv')
#df.describe()
#df.info()
df.head()

# <a id="3"></a> Defining a Function to Calculate the Adjusted $R^{2}$
#### [Return Contents](#0)
<hr/>

The R-squared increases when the number of features increase. Because of this, sometimes a more robust evaluator is preferred to compare the performance between different models. This evaluater is called adjusted R-squared and it only increases, if the addition of the variable reduces the MSE. The definition of the adjusted $R^{2}$ is:

$\bar{R^{2}}=R^{2}-\frac{k-1}{n-k}(1-R^{2})$,

where n is the number of observations and k is the number of parameters. 

In [ ]:
def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)

# <a id="4"></a> Creating a Simple Regression 
#### [Return Contents](#0)
<hr/>

When we model a linear relationship between a response and just **one** explanatory variable, this is called **simple linear regression**. I want to predict house prices and then, our response variable is price. However, for a simple model we also need to select a feature. When I look at the columns of the dataset, **living area (sqft)** seemed the most important feature. When we examine the  <a href=#7>correlation matrix</a>, we may observe that price has the highest correlation coefficient with living area (sqft) and this also supports my opinion. Thus, I decided to use **living area (sqft)** as feature but if you want to examine the relationship between price and another feature, you may prefer the other feature.

In [ ]:
# splitting data
train_data,test_data = train_test_split(df,train_size = 0.8,random_state=3)
# Linear Model 
lr = linear_model.LinearRegression()
X_train = np.array(train_data['sqft_living'], dtype=pd.Series).reshape(-1,1)
y_train = np.array(train_data['price'], dtype=pd.Series)
# fitting the linear model
lr.fit(X_train,y_train)

# Evaluate the simple model
X_test = np.array(test_data['sqft_living'], dtype=pd.Series).reshape(-1,1)
y_test = np.array(test_data['price'], dtype=pd.Series)

pred = lr.predict(X_test)
msesm = format(np.sqrt(metrics.mean_squared_error(y_test,pred)),'.3f')
rtrsm = format(lr.score(X_train, y_train),'.3f')
rtesm = format(lr.score(X_test, y_test),'.3f')

print ("Average Price for Test Data: {:.3f}".format(y_test.mean()))
print('Intercept: {}'.format(lr.intercept_))
print('Coefficient: {}'.format(lr.coef_))

r = evaluation.shape[0]
evaluation.loc[r] = ['Simple Model','-',msesm,rtrsm,'-',rtesm,'-']
evaluation

## <a id="5"></a> Let's Show the Results

Because we have just **two** dimensions at the simple regression, it is easy to determine it. The below chart determines the result of the simple regression. It does not look like a perfect fit but when we work with real world datasets, having a perfect fit is not easy.

In [ ]:
plt.figure(figsize=(6.5,5))
plt.scatter(X_test,y_test,color='darkgreen',label="Data", alpha=.1)
plt.plot(X_test,lr.predict(X_test),color="red",label="Predicted Regression Line")
plt.xlabel("Living Space (sqft)", fontsize=15)
plt.ylabel("Price ($)", fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.legend()

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

# <a id="6"></a> Examining and Adding More Features
#### [Return Contents](#0)
<hr/>

In the previous section I used a simple linear regression and found a poor fit. In order to improve this model I am planing to add more features but we should be careful about the overfit which can be seen by the difference between the training and test evaluation metrics. When we have more than one feature in a linear regression, it is defined as multiple regression. Then, it is time to check the correlation matrix before fitting a mutiple regression.

In [ ]:
sns.set(style="white", font_scale=1)

## <a id="7"></a> Checking Out the Correlation Among Explanatory Variables

First, I want to recall some information. Having too many features in a model is not always a good thing because it might cause overfit and worser results when we want to predict values for a new dataset. Thus, if a feature does not improve your model a lot, not adding it may be a better choice.

Another important thing is correlation, if there is very high correlation between two features, keeping both of them is not a good idea most of the time. For instance, sqt_above and sqt_living are highly correlated. This can be estimated when you look at the definitions at the dataset and checked to be sure by looking at the correlation matrix. However, this does not mean that you should remove one of the highly correlated features. For instance: bathrooms and sqrt_living. They are highly correlated but I do not think that the relation among them is the same as the relation between sqt_living and sqt_above.

In [ ]:
features = ['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront',
            'view','condition','grade','sqft_above','sqft_basement','yr_built',
            'yr_renovated','zipcode','sqft_living15','sqft_lot15']

mask = np.zeros_like(df[features].corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = True 

f, ax = plt.subplots(figsize=(16, 12))
plt.title('Pearson Correlation Matrix',fontsize=25)

sns.heatmap(df[features].corr(),linewidths=0.25,vmax=1.0,square=True,cmap="BuGn_r", 
            linecolor='w',annot=True,mask=mask,cbar_kws={"shrink": .75})

## <a id="8"></a> Complex Model - 1
#### [Return Contents](#0)

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.boxplot(x=train_data['bedrooms'],y=train_data['price'], ax=axes[0])
sns.boxplot(x=train_data['floors'],y=train_data['price'], ax=axes[1])
axes[0].set(xlabel='Bedrooms', ylabel='Price')
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='Floors', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(12.18,5))
sns.boxplot(x=train_data['bathrooms'],y=train_data['price'], ax=axe)
axe.set(xlabel='Bathrooms / Bedrooms', ylabel='Price')

In [ ]:
fig=plt.figure(figsize=(19,12.5))
ax=fig.add_subplot(2,2,1, projection="3d")
ax.scatter(train_data['floors'],train_data['bedrooms'],train_data['bathrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nFloors',ylabel='\nBedrooms',zlabel='\nBathrooms / Bedrooms')
ax.set(ylim=[0,12])

ax=fig.add_subplot(2,2,2, projection="3d")
ax.scatter(train_data['floors'],train_data['bedrooms'],train_data['sqft_living'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nFloors',ylabel='\nBedrooms',zlabel='\nsqft Living')
ax.set(ylim=[0,12])

ax=fig.add_subplot(2,2,3, projection="3d")
ax.scatter(train_data['sqft_living'],train_data['sqft_lot'],train_data['bathrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\n sqft Living',ylabel='\nsqft Lot',zlabel='\nBathrooms / Bedrooms')
ax.set(ylim=[0,250000])

ax=fig.add_subplot(2,2,4, projection="3d")
ax.scatter(train_data['sqft_living'],train_data['sqft_lot'],train_data['bedrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\n sqft Living',ylabel='\nsqft Lot',zlabel='Bedrooms')
ax.set(ylim=[0,250000])

In [ ]:
# Linear model with more features
features1 = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','zipcode']
complex_model_1 = linear_model.LinearRegression()
complex_model_1.fit(train_data[features1],train_data['price'])

print('Intercept: {}'.format(complex_model_1.intercept_))
print('Coefficients: {}'.format(complex_model_1.coef_))

pred1 = complex_model_1.predict(test_data[features1])
msecm1 = format(np.sqrt(metrics.mean_squared_error(y_test,pred1)),'.3f')
rtrcm1 = format(complex_model_1.score(train_data[features1],train_data['price']),'.3f')
artrcm1 = format(adjustedR2(complex_model_1.score(train_data[features1],train_data['price']),train_data.shape[0],len(features1)),'.3f')
rtecm1 = format(complex_model_1.score(test_data[features1],test_data['price']),'.3f')
artecm1 = format(adjustedR2(complex_model_1.score(test_data[features1],test_data['price']),test_data.shape[0],len(features1)),'.3f')

r = evaluation.shape[0]
evaluation.loc[r] = ['Complex Model-1','-',msecm1,rtrcm1,artrcm1,rtecm1,artecm1]
evaluation.sort_values(by = 'R-squared (test)', ascending=False)

## <a id="9"></a> Complex Model - 2
#### [Return Contents](#0)

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.boxplot(x=train_data['waterfront'],y=train_data['price'], ax=axes[0])
sns.boxplot(x=train_data['view'],y=train_data['price'], ax=axes[1])
axes[0].set(xlabel='Waterfront', ylabel='Price')
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='View', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(12.18,5))
sns.boxplot(x=train_data['grade'],y=train_data['price'], ax=axe)
axe.set(xlabel='Grade', ylabel='Price')

In [ ]:
fig=plt.figure(figsize=(9.5,6.25))
ax=fig.add_subplot(1,1,1, projection="3d")
ax.scatter(train_data['view'],train_data['grade'],train_data['yr_built'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nFloors',ylabel='\nBedrooms',zlabel='\n Year Built')

In [ ]:
features2 = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view',
             'grade','yr_built','zipcode']
complex_model_2 = linear_model.LinearRegression()
complex_model_2.fit(train_data[features2],train_data['price'])

print('Intercept: {}'.format(complex_model_2.intercept_))
print('Coefficients: {}'.format(complex_model_2.coef_))

pred2 = complex_model_2.predict(test_data[features2])
msecm2 = format(np.sqrt(metrics.mean_squared_error(y_test,pred2)),'.3f')
rtrcm2 = format(complex_model_2.score(train_data[features2],train_data['price']),'.3f')
artrcm2 = format(adjustedR2(complex_model_2.score(train_data[features2],train_data['price']),train_data.shape[0],len(features2)),'.3f')
rtecm2 = format(complex_model_2.score(test_data[features2],test_data['price']),'.3f')
artecm2 = format(adjustedR2(complex_model_2.score(test_data[features2],test_data['price']),test_data.shape[0],len(features2)),'.3f')

r = evaluation.shape[0]
evaluation.loc[r] = ['Complex Model-2','-',msecm2,rtrcm2,artrcm2,rtecm2,artecm2]
evaluation.sort_values(by = 'R-squared (test)', ascending=False)

# <a id="10"></a> Polynomial Regression
#### [Return Contents](#0)
<hr/>

In [ ]:
polyfeat = PolynomialFeatures(degree=2)
X_trainpoly = polyfeat.fit_transform(train_data[features2])
X_testpoly = polyfeat.fit_transform(test_data[features2])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

predp = poly.predict(X_testpoly)
msepoly1 = format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred)),'.3f')
rtrpoly1 = format(poly.score(X_trainpoly,train_data['price']),'.3f')
rtepoly1 = format(poly.score(X_testpoly,test_data['price']),'.3f')

polyfeat = PolynomialFeatures(degree=3)
X_trainpoly = polyfeat.fit_transform(train_data[features2])
X_testpoly = polyfeat.fit_transform(test_data[features2])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

predp = poly.predict(X_testpoly)
msepoly2 = format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred)),'.3f')
rtrpoly2 = format(poly.score(X_trainpoly,train_data['price']),'.3f')
rtepoly2 = format(poly.score(X_testpoly,test_data['price']),'.3f')

r = evaluation.shape[0]
evaluation.loc[r] = ['Polynomial Regression','degree=2',msepoly1,rtrpoly1,'-',rtepoly1,'-']
evaluation.loc[r+1] = ['Polynomial Regression','degree=3',msepoly2,rtrpoly2,'-',rtepoly2,'-']
evaluation.sort_values(by = 'R-squared (test)', ascending=False)

# <a id="11"></a> KNN Regression 
#### [Return Contents](#0)
<hr/>

In [ ]:
knnreg = KNeighborsRegressor(n_neighbors=15)
knnreg.fit(train_data[features2],train_data['price'])
pred = knnreg.predict(test_data[features2])

mseknn1 = format(np.sqrt(metrics.mean_squared_error(y_test,pred)),'.3f')
rtrknn1 = format(knnreg.score(train_data[features2],train_data['price']),'.3f')
artrknn1 = format(adjustedR2(knnreg.score(train_data[features2],train_data['price']),train_data.shape[0],len(features2)),'.3f')
rteknn1 = format(knnreg.score(test_data[features2],test_data['price']),'.3f')
arteknn1 = format(adjustedR2(knnreg.score(test_data[features2],test_data['price']),test_data.shape[0],len(features2)),'.3f')

knnreg = KNeighborsRegressor(n_neighbors=25)
knnreg.fit(train_data[features2],train_data['price'])
pred = knnreg.predict(test_data[features2])

mseknn2 = format(np.sqrt(metrics.mean_squared_error(y_test,pred)),'.3f')
rtrknn2 = format(knnreg.score(train_data[features2],train_data['price']),'.3f')
artrknn2 = format(adjustedR2(knnreg.score(train_data[features2],train_data['price']),train_data.shape[0],len(features2)),'.3f')
rteknn2 = format(knnreg.score(test_data[features2],test_data['price']),'.3f')
arteknn2 = format(adjustedR2(knnreg.score(test_data[features2],test_data['price']),test_data.shape[0],len(features2)),'.3f')


knnreg = KNeighborsRegressor(n_neighbors=27)
knnreg.fit(train_data[features2],train_data['price'])
pred = knnreg.predict(test_data[features2])

mseknn3 = format(np.sqrt(metrics.mean_squared_error(y_test,pred)),'.3f')
rtrknn3 = format(knnreg.score(train_data[features2],train_data['price']),'.3f')
artrknn3 = format(adjustedR2(knnreg.score(train_data[features2],train_data['price']),train_data.shape[0],len(features2)),'.3f')
rteknn3 = format(knnreg.score(test_data[features2],test_data['price']),'.3f')
arteknn3 = format(adjustedR2(knnreg.score(test_data[features2],test_data['price']),test_data.shape[0],len(features2)),'.3f')

r = evaluation.shape[0]
evaluation.loc[r] = ['KNN Regression','n_neighbors=15',mseknn1,rtrknn1,artrknn1,rteknn1,arteknn1]
evaluation.loc[r+1] = ['KNN Regression','n_neighbors=25',mseknn2,rtrknn2,artrknn2,rteknn2,arteknn2]
evaluation.loc[r+2] = ['KNN Regression','n_neighbors=27',mseknn3,rtrknn3,artrknn3,rteknn3,arteknn3]
evaluation.sort_values(by = 'R-squared (test)', ascending=False)

# <a id="12"></a> Conclusion
#### [Return Contents](#0)
<hr/>
When we look at the evaluation table, **Polynomial Regression** is the best. However, each model might be useful depending to the situation.